In [6]:
# In the name of Allah

In [41]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from sklearn.model_selection import GridSearchCV
import keras.backend as K
import tensorflow as tf
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from collections import Counter
import joblib
import pickle

## Data Preprocessing

In [8]:
df = pd.read_csv('crf_file_real.csv', encoding = "ISO-8859-1")
# df = df[:10000]
df = df.fillna(method='ffill')
df.head()

,Sentence #,Word,Tag
0,Sentence: 1,Another,A
1,Sentence: 1,violent,P
2,Sentence: 1,and,P
3,Sentence: 1,aggressive,P
4,Sentence: 1,immigrant,P


In [9]:
embeddings = {}
oneDEmbeddings={}
oneDTotal=[]
def extractEmbeddings():
    with open('glove.6B.50d.txt',encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coeffs = np.asarray(values[1:],dtype='float32')
            #print(word)
            #print(coeffs)
            embeddings[word] = coeffs
            oneDTotal.append(coeffs)
    return embeddings

In [10]:
def getEmbeddings(word):
    a=np.zeros(50)
    if word not in embeddings:
        return a
    return embeddings[word]

In [11]:
def getEmbeddings2(word):
    a=0
    if word not in oneDEmbeddings:
        return a
    return oneDEmbeddings[word][0]

In [12]:
def store(df,neg,pos):
    n,m=df.shape
    for i in range(0,n):
        word=df.iloc[i,1]
        label=df.iloc[i,2]
        wordEmbedding=getEmbeddings2(word)
        if label=='A':
            neg.append(wordEmbedding)
        else:
            pos.append(wordEmbedding)

In [19]:
# extractEmbeddings()

In [14]:
# print(len(embeddings))
# print(type(embeddings))
# print(embeddings["because"])
# print(oneDTotal[0])

In [15]:
from sklearn.decomposition import PCA
def reduce_dimension(x):
    pca = PCA(n_components=1)
    z= pca.fit_transform(x)
    return z
# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])

In [16]:
oneDTotal=reduce_dimension(oneDTotal)
# print(oneDTotal[0])

In [17]:
i=0
for key in embeddings.keys():
    oneDEmbeddings[key]=oneDTotal[i]
    i+=1

In [18]:
neg=[]
pos=[]
store(df,neg,pos)
pos.sort()
neg.sort()

In [20]:
import pickle
with open('pos.pkl', 'wb') as f:
    pickle.dump(pos, f)
    
with open('neg.pkl', 'wb') as f:
    pickle.dump(neg,f)
    
a_file = open("oneDEmbeddings.pkl", "wb")
pickle.dump(oneDEmbeddings, a_file)
a_file.close()

a_file = open("embeddings.pkl", "wb")
pickle.dump(embeddings, a_file)
a_file.close()

In [21]:
print(len(neg))
print(len(pos))

274071
20590


In [22]:
# for i in range (0,10):
#     print(pos[i])
#     print(neg[i])
#     print()

In [23]:
def extractSentences(X,Y,df):
        n,m=df.shape
        tmpX=[]
        tmpY=[]
        for i in range(0,n):
            if i==0 or df.iloc[i,0]!=df.iloc[i-1,0]:
                if len(tmpX)>1:
                    tmpX.pop()
                    tmpY.pop()
                    X.append(tmpX)
                    Y.append(tmpY)
                    tmpX=[]
                    tmpY=[]
            tmpX.append(df.iloc[i,1])
            tmpY.append(df.iloc[i,2])
        if len(tmpX)>1:
            tmpX.pop()
            tmpY.pop()
            X.append(tmpX)
            Y.append(tmpY)

In [24]:
X,Y=[],[]
extractSentences(X,Y,df)

In [25]:
from numpy import dot
from numpy.linalg import norm

In [26]:
def cossim(wordEmbedding,pos):
    p1=pos
    p1=p1.reshape(-1)
    w1=wordEmbedding
    w1=w1.reshape(-1)
    if norm(p1)==0 or norm(w1)==0:
        return 0
    val=dot(w1,p1)/(norm(w1)*norm(p1))
    return val

In [27]:
def getKNearest(wordEmbedding,pos,tmp,k):
    v=[]
    for i in range(0,len(pos)):
        v.append(cossim(wordEmbedding,pos[i]))
        
    v.sort(reverse=True)
    for i in range(0,k):
        tmp.append(v[i])

In [28]:
def getKNearest2(oneDwordEmbedding,pos,tmp,k):
    low,high=0,len(pos)-1
    ans=0
    while low<=high:
        mid=int((low+high)/2)
        if pos[mid]==oneDwordEmbedding:
            ans=mid
            break
        elif pos[mid]<oneDwordEmbedding:
            ans=mid
            low=mid+1
        else:
            high=mid-1
        
    cnt,i,j=0,ans-1,ans+1
    while cnt<10 and i>=0 and j<len(pos):
        cnt+=1
        if abs(pos[i]-oneDwordEmbedding)<=abs(pos[j]-oneDwordEmbedding):
            tmp.append(pos[i])
            i-=1
        else:
            tmp.append(pos[j])
            j+=1
                
        while cnt<10 and j<len(pos):
            cnt+=1
            tmp.append(pos[j])
            j+=1
            
        while cnt<10 and i>=0:
            cnt+=1
            tmp.append(pos[i])
            i-=1

In [29]:
def store_dataset2(dataset,X,Y,pos,neg,k1):
    n=len(X)
    for i in range(0,n):
#         print(i,n)
#         if i==1:
#             break
        for j in range(0,len(X[i])):
            tmp=[]
            word=X[i][j].lower()
            label=0 if Y[i][j]=='A' else 1
            wordEmbedding=getEmbeddings(word)
            oneDwordEmbedding=getEmbeddings2(word)
            #print(word,wordEmbedding,label)
            getKNearest2(oneDwordEmbedding,pos,tmp,k1)
#             print(tmp)
#             print("amir")
            getKNearest2(oneDwordEmbedding,neg,tmp,k1)
#             print(tmp)
            for k in range(j-1,max(-1,j-6),-1):
                l=0 if Y[i][k]=='A' else 1
                newEmbedding=getEmbeddings(X[i][k].lower())
                tmp.append(cossim(wordEmbedding,newEmbedding))
                tmp.append(l)
            while len(tmp)<2*k1+10:
                tmp.append(0)
            tmp.append(label)
            #print(len(tmp),word)
            dataset.append(tmp)

In [30]:
def store_dataset(dataset,X,Y,pos,neg,k1):
    n=len(X)
    for i in range(0,n):
#         print(i,n)
        if i==10:
            break
        for j in range(0,len(X[i])):
            tmp=[]
            word=X[i][j].lower()
            label=0 if Y[i][j]=='A' else 1
            wordEmbedding=getEmbeddings(word)
            #print(word,wordEmbedding,label)
            getKNearest(wordEmbedding,pos,tmp,k1)
            getKNearest(wordEmbedding,neg,tmp,k1)
            for k in range(j-1,max(-1,j-6),-1):
                l=0 if Y[i][k]=='A' else 1
                newEmbedding=getEmbeddings(X[i][k].lower())
                tmp.append(cossim(wordEmbedding,newEmbedding))
                tmp.append(l)
            while len(tmp)<2*k1+10:
                tmp.append(0)
            tmp.append(label)
            #print(len(tmp),word)
            dataset.append(tmp)
            
            

In [31]:
dataset=[]
store_dataset2(dataset,X,Y,pos,neg,10)

In [32]:
# for i in range(0,3):
#     print(dataset[i])

In [34]:
def saveFile(fileName,data):
    #savetxt('data.csv', data, delimiter=',')
    df=pd.DataFrame(data)
    df.to_csv(fileName+".csv", header=None, index=None)

In [35]:
#saveFile("XEmbedded",XEmbedded)
saveFile("ANN_Dataset",dataset)

In [37]:
df.head(50)
print(type(dataset))
dataset=np.array(dataset)
print(type(dataset))

<class 'list'>
<class 'numpy.ndarray'>


In [39]:
X_train=dataset[:,:-1]
y_train=dataset[:,-1]

In [42]:
print(len(X_train))
print(len(Y_train))
print(Counter(y_train))

286722
286722
Counter({0.0: 266132, 1.0: 20590})


In [43]:
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE().fit_resample(X_train, y_train)
#plot_2d_space(X_resampled, y_resampled, 'SMOTE over-sampling')

In [44]:
print(len(X_train))
print(len(y_train))
print(Counter(y_train))

532264
532264
Counter({0.0: 266132, 1.0: 266132})


In [45]:
with open('X_train_real.pkl', 'wb') as f:
    pickle.dump(X_train, f)
    
with open('y_train_real.pkl', 'wb') as f:
    pickle.dump(y_train, f)

In [46]:
with open('X_train_real.pkl', 'rb') as f:
        X_train = pickle.load(f)
with open('y_train_real.pkl', 'rb') as f:
    y_train_real = pickle.load(f)


In [35]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3,random_state=109) # 70% training and 30% test

## Naive Bayes :- Model Generation

In [36]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

## Evaluating Model

In [37]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7973990417522245


In [38]:
# Save the model as a pickle in a file
joblib.dump(gnb, 'naive_bayes_model.pkl')

['naive_bayes_model.pkl']

## Random Forest :- Model Generation

In [39]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [40]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9582477754962354


In [41]:
# Save the model as a pickle in a file
joblib.dump(clf, 'random_forest_model.pkl')

['random_forest_model.pkl']

## Deep neural network :- Model Generation

In [42]:
#!pip install keras
#!pip install tensorflow

In [43]:
# first neural network with keras tutorial


In [44]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [45]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=30, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [46]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])

In [47]:
#!pip install imblearn
#!pip install delayed

In [48]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
#plot_2d_space(X_resampled, y_resampled, 'SMOTE over-sampling')

In [49]:
# fit the keras model on the dataset
model.fit(X_resampled, y_resampled, epochs=10, batch_size=10)

Epoch 1/10
1295/1295 [==============================] - 15s 9ms/step - loss: 0.5790 - accuracy: 0.6895 - f1: 0.6280
Epoch 2/10
1295/1295 [==============================] - 8s 6ms/step - loss: 0.3993 - accuracy: 0.8189 - f1: 0.7984
Epoch 3/10
1295/1295 [==============================] - 9s 7ms/step - loss: 0.3917 - accuracy: 0.8298 - f1: 0.8176
Epoch 4/10
1295/1295 [==============================] - 8s 6ms/step - loss: 0.3948 - accuracy: 0.8292 - f1: 0.8180
Epoch 5/10
1295/1295 [==============================] - 8s 6ms/step - loss: 0.3917 - accuracy: 0.8296 - f1: 0.8177
Epoch 6/10
1295/1295 [==============================] - 9s 7ms/step - loss: 0.3829 - accuracy: 0.8348 - f1: 0.8239
Epoch 7/10
1295/1295 [==============================] - 10s 8ms/step - loss: 0.3866 - accuracy: 0.8296 - f1: 0.8239
Epoch 8/10
1295/1295 [==============================] - 11s 8ms/step - loss: 0.3777 - accuracy: 0.8361 - f1: 0.8260
Epoch 9/10
1295/1295 [==============================] - 9s 7ms/step - loss: 0

In [52]:
# evaluate the keras model
_, accuracy,f1_score = model.evaluate(X_test, y_test)
print('Accuracy: %.2f %.2f' % (accuracy*100,f1_score))

92/92 [==============================] - 1s 7ms/step - loss: 0.4761 - accuracy: 0.7645 - f1: 0.2630
Accuracy: 76.45 0.26


In [58]:
model.save("ann_model")

INFO:tensorflow:Assets written to: ann_model/assets


In [64]:
model.save("ann_model.h5")

In [68]:
tmp=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(len(tmp))
model.predict([tmp])
print(type(tmp))

30
<class 'list'>
